In [1]:
logg = lambda x: print(f"------------------------ {x} ---------------------------")

import os
import gc
import torch
import transformers
import huggingface_hub
import wandb

wandb.require("core")
from datetime import datetime
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import SFTTrainer
from peft import LoraConfig, PeftModel
from datasets import load_dataset
from time import time
load_dotenv()
from utils.eval_helper import *



In [2]:
cache_dir = f"/dpc/kunf0097/l3-8b"
candidate_name = "amew0/l3-8b-medical-v240623023136"
candidate_tokenizer, candidate_model = get_tokenizer_and_model(
    model_name=candidate_name, cache_dir=cache_dir
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
!curl ipv4.icanhazip.com

109.205.28.10


In [4]:
!python -m vllm.entrypoints.openai.api_server --host 127.0.0.1 --port 8888 --model amew0/l3-8b-medical-v240622193618 --dtype float16 --download-dir /dpc/kunf0097/l3-8b/model

/home/kunet.ae/ku5001069/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
INFO 07-09 14:42:30 llm_engine.py:72] Initializing an LLM engine with config: model='amew0/l3-8b-medical-v240622193618', tokenizer='amew0/l3-8b-medical-v240622193618', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir='/dpc/kunf0097/l3-8b/model', load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)
Traceback (most recent call last):
  File "/home/kunet.ae/ku5001069/.conda/envs/torch20/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/kunet.ae/ku5001069/.conda/envs/torch20/lib/python3.10/runpy.py", line 86, in _run_c

In [58]:

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    candidate_name,
    cache_dir=f"{cache_dir}/model",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map={"": 0},
    low_cpu_mem_usage=True,
    return_dict=True,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [79]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaR

In [80]:

for i, module in enumerate(model.modules()):
    a=module.model.layers[0].self_attn.q_proj.weight
    print(a.shape)
    # print(i)
    break

torch.Size([8388608, 1])


In [89]:
(8388608/(4096*4096))**-1

2.0

In [81]:
for i, module in enumerate(candidate_model.modules()):
    a=module.model.layers[0].self_attn.q_proj.weight
    print(a.shape)
    break

torch.Size([4096, 4096])


In [28]:
candidate_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Li